##**Librerias y dependencias necesarias**


In [ ]:
from google.colab import drive #Acceso Drive
import os #Acceder a directorios
import pandas as pd #manejo de dataframes
from IPython.display import Audio #reproducir audio
import folium #mostrar mapa con geocolocacion 
import numpy as np #Numerical Python

##**Acceso al drive y ubicacion archivos**

In [ ]:
#Damos acceso a colab a nuestro Drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


**Debemos asegurarnos de que la carpeta de las grabaciones compartida tenga un acceso directo creado en nuestro propio Drive.**

In [ ]:
#Nos ubicamos en la direccion de interes
#Alli estaran las imagente fuente y las destino
ruta_principal='/content/drive/MyDrive/Jaguas_2018' #direccion raiz de nuestro proyecto actual
os.chdir(ruta_principal) #Ubicarnos en el directorio
print("Ubicados en :",dir)
!ls

Ubicados en : <built-in function dir>
G03_m  G07_m  G13_m  G19_m  G25_m  G29_m  G36_m  G41_m	G49_m  G52_m  G58_m
G04_m  G08_m  G15_m  G23_m  G27_m  G34_m  G37_m  G46_m	G50_m  G54_m
G06_m  G09_m  G17_m  G24_m  G28_m  G35_m  G40_m  G47_m	G51_m  G57_m


In [ ]:
"""Cuenta la cantidad de archivos en cada carpeta de un directorio"""
total_archivos = 0
for raiz, carpetas, archivos in os.walk(ruta_principal):
  carpetas.sort()
  # Obtener la cantidad de archivos en la carpeta actual
  cantidad_archivos = len(archivos)  
  # Sumar la cantidad de archivos actual a la variable total_archivos
  total_archivos += cantidad_archivos      
  # Imprimir el resultado
  print(f"Grabadora: {os.path.basename(raiz)} Cantidad Grabaciones: {cantidad_archivos}")
print(f"Total grabaciones de todas las grabadoras: {total_archivos}")

Grabadora: Jaguas_2018 Cantidad Grabaciones: 0
Grabadora: G03_m Cantidad Grabaciones: 565
Grabadora: G04_m Cantidad Grabaciones: 418
Grabadora: G06_m Cantidad Grabaciones: 887
Grabadora: G07_m Cantidad Grabaciones: 621
Grabadora: G08_m Cantidad Grabaciones: 648
Grabadora: G09_m Cantidad Grabaciones: 908
Grabadora: G13_m Cantidad Grabaciones: 636
Grabadora: G15_m Cantidad Grabaciones: 874
Grabadora: G17_m Cantidad Grabaciones: 870
Grabadora: G19_m Cantidad Grabaciones: 886
Grabadora: G23_m Cantidad Grabaciones: 555
Grabadora: G24_m Cantidad Grabaciones: 555
Grabadora: G25_m Cantidad Grabaciones: 645
Grabadora: G27_m Cantidad Grabaciones: 644
Grabadora: G28_m Cantidad Grabaciones: 553
Grabadora: G29_m Cantidad Grabaciones: 553
Grabadora: G34_m Cantidad Grabaciones: 664
Grabadora: G35_m Cantidad Grabaciones: 627
Grabadora: G36_m Cantidad Grabaciones: 545
Grabadora: G37_m Cantidad Grabaciones: 541
Grabadora: G40_m Cantidad Grabaciones: 555
Grabadora: G41_m Cantidad Grabaciones: 553
Grabado

##**Creando el Data Frame de los Audios**

In [ ]:
# Crear un dataframe vacío para guardar la información de los archivos
df_audios = pd.DataFrame(columns=['Recording','Recorder', 'Date', 'Time', 'Path'])

# Recorrer las carpetas y archivos dentro de la ruta principal
for raiz, carpetas, archivos in os.walk(ruta_principal):
    carpetas.sort()
    # Recorrer los archivos de cada carpeta
    for archivo in archivos:
        # Obtener la ruta completa del archivo
        ruta_archivo = os.path.join(raiz, archivo)
        
        # Obtener el nombre del archivo sin la extensión
        nombre_archivo, extension = os.path.splitext(archivo)
        
        # Separar el nombre del archivo en partes utilizando el caracter "_"
        partes_nombre = nombre_archivo.split("_")
        
        # Obtener los valores deseados de cada parte del nombre del archivo
        recorder = partes_nombre[0]
        fecha = partes_nombre[1]
        hora = partes_nombre[2]
        
        # Crear un diccionario con los valores del archivo actual
        datos_archivo = {
            'Recording': nombre_archivo,
            'Recorder': recorder,
            'Date': fecha,
            'Time': hora,
            'Path': ruta_archivo
        }
        
        # Agregar el diccionario al dataframe
        df_audios = df_audios.append(datos_archivo, ignore_index=True)


# Convertir la columna 'Date' al formato 'yyyy/mm/dd'
df_audios['Date'] = df_audios['Date'].apply(lambda x: pd.to_datetime(x, format='%Y%m%d').strftime('%Y/%m/%d'))

# Convertir la columna 'Time' al formato 'hh:mm:ss'
df_audios['Time'] = df_audios['Time'].apply(lambda x: pd.to_datetime(x, format='%H%M%S').strftime('%H:%M:%S'))

#Cargamos informacion adicional de las grabadoras
df_grabadoras = pd.read_csv('/content/drive/MyDrive/Bio_Acustica/df_grabadoras.csv')

#hacemos un Marge con el dataframe anteriormente creado, a base de recorrer los archivos
df_audios = pd.merge(df_audios, df_grabadoras, on="Recorder")

In [ ]:
df_audios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20068 entries, 0 to 20067
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Recording  20068 non-null  object 
 1   Recorder   20068 non-null  object 
 2   Date       20068 non-null  object 
 3   Time       20068 non-null  object 
 4   Path       20068 non-null  object 
 5   Municipio  20068 non-null  object 
 6   Vereda     20068 non-null  object 
 7   Latitud    20068 non-null  float64
 8   Longitud   20068 non-null  float64
 9   Elevacion  20068 non-null  int64  
 10  Habitat    20068 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 1.7+ MB


In [ ]:
df_audios

,Recording,Recorder,Date,Time,Path,Municipio,Vereda,Latitud,Longitud,Elevacion,Habitat
0,G03_20180511_120000,G03,2018/05/11,12:00:00,/content/drive/MyDrive/Jaguas_2018/G03_m/G03_2...,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest
1,G03_20180511_121600,G03,2018/05/11,12:16:00,/content/drive/MyDrive/Jaguas_2018/G03_m/G03_2...,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest
2,G03_20180511_123200,G03,2018/05/11,12:32:00,/content/drive/MyDrive/Jaguas_2018/G03_m/G03_2...,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest
3,G03_20180511_124800,G03,2018/05/11,12:48:00,/content/drive/MyDrive/Jaguas_2018/G03_m/G03_2...,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest
4,G03_20180511_130400,G03,2018/05/11,13:04:00,/content/drive/MyDrive/Jaguas_2018/G03_m/G03_2...,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest
...,...,...,...,...,...,...,...,...,...,...,...
20063,G58_20180704_070000,G58,2018/07/04,07:00:00,/content/drive/MyDrive/Jaguas_2018/G58_m/G58_2...,San Rafael,Area proteccion embalse EPM,6.3471,-74.9696,1072,Forest
20064,G58_20180704_064500,G58,2018/07/04,06:45:00,/content/drive/MyDrive/Jaguas_2018/G58_m/G58_2...,San Rafael,Area proteccion embalse EPM,6.3471,-74.9696,1072,Forest
20065,G58_20180704_071500,G58,2018/07/04,07:15:00,/content/drive/MyDrive/Jaguas_2018/G58_m/G58_2...,San Rafael,Area proteccion embalse EPM,6.3471,-74.9696,1072,Forest
20066,G58_20180704_073000,G58,2018/07/04,07:30:00,/content/drive/MyDrive/Jaguas_2018/G58_m/G58_2...,San Rafael,Area proteccion embalse EPM,6.3471,-74.9696,1072,Forest


In [ ]:
df_audios["Recorder"].value_counts().sort_index()

G03    565
G04    418
G06    887
G07    621
G08    648
G09    908
G13    636
G15    874
G17    870
G19    886
G23    555
G24    555
G25    645
G27    644
G28    553
G29    553
G34    664
G35    627
G36    545
G37    541
G40    555
G41    553
G46    735
G47    804
G49    801
G50    620
G51    408
G52    465
G54    576
G57    621
G58    735
Name: Recorder, dtype: int64

In [ ]:
# Crea un DataFrame a partir del resultado de value_counts()
df_audios_counts = pd.DataFrame(df_audios["Recorder"].value_counts().sort_index())

# Reinicia el índice para convertir el índice en una columna
df_audios_counts = df_audios_counts.reset_index()

# Renombra las columnas para que coincidan con las del DataFrame df_grabadoras
df_audios_counts.columns = ["Recorder", "Total_grabaciones"]

# Fusiona los dos DataFrames en función de la columna "Recorder"
df_grabadoras = df_grabadoras.merge(df_audios_counts, on="Recorder")

df_grabadoras


,Recorder,Municipio,Vereda,Latitud,Longitud,Elevacion,Habitat,Total_grabaciones
0,G03,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest,565
1,G04,San Roque,Playa Rica,6.4286,-75.0281,1310,Forest,418
2,G06,Alejandria,El Respaldo,6.4088,-75.0453,1302,Forest,887
3,G07,San Roque,El Nucito,6.4287,-75.0368,1296,Non- forest,621
4,G08,Alejandria,El Respaldo,6.4061,-75.0534,1314,Non- forest,648
5,G09,Santo Domingo,Nusito-Dantas,6.4204,-75.0474,1346,Forest,908
6,G13,Alejandria,El Respaldo,6.3968,-75.0186,1349,Forest,636
7,G15,San Roque,Playa Rica,6.4045,-75.0143,1325,Forest,874
8,G17,Alejandria,El Respaldo,6.3978,-75.0444,1389,Non- forest,870
9,G19,Alejandria,El Respaldo,6.4075,-75.0308,1376,Non- forest,886


In [ ]:
# Crear un dataframe vacío para guardar la información de los cluster
df_audios_cluster = pd.DataFrame(columns=['Recording', 'Cluster'])

# Recorrer las carpetas y archivos dentro de la ruta principal
for raiz, carpetas, archivos in os.walk(ruta_principal):
    carpetas.sort()
    # Recorrer los archivos de cada carpeta
    for archivo in archivos:
        # Obtener la ruta completa del archivo
        ruta_archivo = os.path.join(raiz, archivo)
        
        # Obtener el nombre del archivo sin la extensión
        nombre_archivo, extension = os.path.splitext(archivo)
        
        # Asignar el nombre del archivo a la columna Recording del dataframe
        df_audios_cluster = df_audios_cluster.append({'Recording': nombre_archivo}, ignore_index=True)
        
        # Asignar un número aleatorio de 1 a 3 a la columna Cluster del dataframe
        df_audios_cluster.loc[df_audios_cluster.index[-1], 'Cluster'] = np.random.choice([1, 2, 3], p=[0.72, 0.27, 0.01])



In [ ]:
df_audios_cluster

,Recording,Cluster
0,G03_20180511_120000,2
1,G03_20180511_121600,1
2,G03_20180511_123200,2
3,G03_20180511_124800,1
4,G03_20180511_130400,1
...,...,...
20063,G58_20180704_070000,1
20064,G58_20180704_064500,1
20065,G58_20180704_071500,1
20066,G58_20180704_073000,1


In [ ]:
#Almacenamos df con la informacion de los audios
df_audios.to_csv('/content/drive/MyDrive/Bio_Acustica/df_audios.csv', index=False)

#Almacenamos df con la informacion de los audios
df_grabadoras.to_csv('/content/drive/MyDrive/Bio_Acustica/df_grabadoras.csv', index=False)

#Almacenamos df con la informacion de los cluster
df_audios_cluster.to_csv('/content/drive/MyDrive/Bio_Acustica/df_audios_cluster.csv', index=False)

#**Cargar Data Frames**


In [ ]:
#Cargamos df con informacion de todos los audios
df_audios = pd.read_csv('/content/drive/MyDrive/Bio_Acustica/df_audios.csv')

#Cargamos df con informacion de la ubicacion de las grabadoras
df_grabadoras = pd.read_csv('/content/drive/MyDrive/Bio_Acustica/df_grabadoras.csv')

#Cargamos df con informacion de la ubicacion de las grabadoras
df_audios_cluster = pd.read_csv('/content/drive/MyDrive/Bio_Acustica/df_audios_cluster.csv')

In [ ]:
df_audios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20068 entries, 0 to 20067
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Recording  20068 non-null  object 
 1   Recorder   20068 non-null  object 
 2   Date       20068 non-null  object 
 3   Time       20068 non-null  object 
 4   Path       20068 non-null  object 
 5   Municipio  20068 non-null  object 
 6   Vereda     20068 non-null  object 
 7   Latitud    20068 non-null  float64
 8   Longitud   20068 non-null  float64
 9   Elevacion  20068 non-null  int64  
 10  Habitat    20068 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 1.7+ MB


In [ ]:
df_audios

,Recording,Recorder,Date,Time,Path,Municipio,Vereda,Latitud,Longitud,Elevacion,Habitat
0,G03_20180511_120000,G03,2018/05/11,12:00:00,/content/drive/MyDrive/Jaguas_2018/G03_m/G03_2...,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest
1,G03_20180511_121600,G03,2018/05/11,12:16:00,/content/drive/MyDrive/Jaguas_2018/G03_m/G03_2...,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest
2,G03_20180511_123200,G03,2018/05/11,12:32:00,/content/drive/MyDrive/Jaguas_2018/G03_m/G03_2...,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest
3,G03_20180511_124800,G03,2018/05/11,12:48:00,/content/drive/MyDrive/Jaguas_2018/G03_m/G03_2...,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest
4,G03_20180511_130400,G03,2018/05/11,13:04:00,/content/drive/MyDrive/Jaguas_2018/G03_m/G03_2...,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest
...,...,...,...,...,...,...,...,...,...,...,...
20063,G58_20180704_070000,G58,2018/07/04,07:00:00,/content/drive/MyDrive/Jaguas_2018/G58_m/G58_2...,San Rafael,Area proteccion embalse EPM,6.3471,-74.9696,1072,Forest
20064,G58_20180704_064500,G58,2018/07/04,06:45:00,/content/drive/MyDrive/Jaguas_2018/G58_m/G58_2...,San Rafael,Area proteccion embalse EPM,6.3471,-74.9696,1072,Forest
20065,G58_20180704_071500,G58,2018/07/04,07:15:00,/content/drive/MyDrive/Jaguas_2018/G58_m/G58_2...,San Rafael,Area proteccion embalse EPM,6.3471,-74.9696,1072,Forest
20066,G58_20180704_073000,G58,2018/07/04,07:30:00,/content/drive/MyDrive/Jaguas_2018/G58_m/G58_2...,San Rafael,Area proteccion embalse EPM,6.3471,-74.9696,1072,Forest


In [ ]:
df_grabadoras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Recorder           31 non-null     object 
 1   Municipio          31 non-null     object 
 2   Vereda             31 non-null     object 
 3   Latitud            31 non-null     float64
 4   Longitud           31 non-null     float64
 5   Elevacion          31 non-null     int64  
 6   Habitat            31 non-null     object 
 7   Total_grabaciones  31 non-null     int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 2.1+ KB


In [ ]:
df_grabadoras

,Recorder,Municipio,Vereda,Latitud,Longitud,Elevacion,Habitat,Total_grabaciones
0,G03,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest,565
1,G04,San Roque,Playa Rica,6.4286,-75.0281,1310,Forest,418
2,G06,Alejandria,El Respaldo,6.4088,-75.0453,1302,Forest,887
3,G07,San Roque,El Nucito,6.4287,-75.0368,1296,Non- forest,621
4,G08,Alejandria,El Respaldo,6.4061,-75.0534,1314,Non- forest,648
5,G09,Santo Domingo,Nusito-Dantas,6.4204,-75.0474,1346,Forest,908
6,G13,Alejandria,El Respaldo,6.3968,-75.0186,1349,Forest,636
7,G15,San Roque,Playa Rica,6.4045,-75.0143,1325,Forest,874
8,G17,Alejandria,El Respaldo,6.3978,-75.0444,1389,Non- forest,870
9,G19,Alejandria,El Respaldo,6.4075,-75.0308,1376,Non- forest,886


In [ ]:
df_audios_cluster.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20068 entries, 0 to 20067
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Recording  20068 non-null  object
 1   Cluster    20068 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 313.7+ KB


In [ ]:
df_audios_cluster

,Recording,Cluster
0,G03_20180511_120000,2
1,G03_20180511_121600,1
2,G03_20180511_123200,2
3,G03_20180511_124800,1
4,G03_20180511_130400,1
...,...,...
20063,G58_20180704_070000,1
20064,G58_20180704_064500,1
20065,G58_20180704_071500,1
20066,G58_20180704_073000,1


#**Exploracion Base De Datos**

In [ ]:
# Reproducir el archivo .wav
Audio(df_audios["Path"][0], autoplay=True)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
len(df_audios["Recorder"].value_counts().sort_index())

31

##**Ubicación grabadoras en mapa**

In [ ]:
# Crea un objeto de mapa en la ubicación especificada
map = folium.Map(location=[6.39, -75.05], zoom_start=12)

# Itera sobre las filas del dataframe y agrega un marcador para cada una
for index, row in df_grabadoras.iterrows():
    location = [row['Latitud'], row['Longitud']]
    tooltip=row['Recorder']
    popup_text = f"{row['Recorder']}\n{row['Municipio']}_{row['Vereda']}\n Latitud: {row['Latitud']}\n Longitud: {row['Longitud']}\n Elevación: {row['Elevacion']}\n Total Grabaciones: {row['Total_grabaciones']}"
    folium.Marker(location=location, popup=popup_text,tooltip=tooltip).add_to(map)

# Muestra el mapa
map.save("/content/drive/MyDrive/Bio_Acustica/mapa_grabadoras.html")
map


#Clusterizando 

In [ ]:
#Contamos el total, de cada cluster en las grabaciones
print(" total, de cada cluster en las grabaciones \n",df_audios_cluster["Cluster"].value_counts())

#Hacemos merged, del df_audios_cluster y el df_audios 
df_audios_cluster_info = pd.merge(df_audios_cluster, df_audios, on="Recording")

#Contomaos el total de cada cluster, para cada grabadora
print(" total, de cada cluster para cada grabadora \n",df_audios_cluster_info.groupby(["Recorder", "Cluster"]).size())

#cremos un nuevo df_grabadoras_cluster con la informacion del numero de cada cluster para cada grabadora
df_grabadoras_cluster=df_grabadoras.copy()

#añadimos las nuevas columnas de los cluster
df_grabadoras_cluster["Cluster 1"]=0
df_grabadoras_cluster["Cluster 2"]=0
df_grabadoras_cluster["Cluster 3"]=0

A,B,C=0,1,2

for i in df_grabadoras_cluster.index:
  df_grabadoras_cluster.at[i,"Cluster 1"]=df_audios_cluster_info.groupby(["Recorder", "Cluster"]).size()[A]
  df_grabadoras_cluster.at[i,"Cluster 2"]=df_audios_cluster_info.groupby(["Recorder", "Cluster"]).size()[B]
  df_grabadoras_cluster.at[i,"Cluster 3"]=df_audios_cluster_info.groupby(["Recorder", "Cluster"]).size()[C]
  A,B,C=A+3,B+3,C+3

 total, de cada cluster en las grabaciones 
 1    14446
2     5415
3      207
Name: Cluster, dtype: int64
 total, de cada cluster para cada grabadora 
 Recorder  Cluster
G03       1          396
          2          163
          3            6
G04       1          299
          2          115
                    ... 
G57       2          165
          3            6
G58       1          523
          2          207
          3            5
Length: 93, dtype: int64


In [ ]:
df_grabadoras_cluster

,Recorder,Municipio,Vereda,Latitud,Longitud,Elevacion,Habitat,Total_grabaciones,Cluster 1,Cluster 2,Cluster 3
0,G03,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest,565,396,163,6
1,G04,San Roque,Playa Rica,6.4286,-75.0281,1310,Forest,418,299,115,4
2,G06,Alejandria,El Respaldo,6.4088,-75.0453,1302,Forest,887,644,232,11
3,G07,San Roque,El Nucito,6.4287,-75.0368,1296,Non- forest,621,467,148,6
4,G08,Alejandria,El Respaldo,6.4061,-75.0534,1314,Non- forest,648,461,181,6
5,G09,Santo Domingo,Nusito-Dantas,6.4204,-75.0474,1346,Forest,908,664,234,10
6,G13,Alejandria,El Respaldo,6.3968,-75.0186,1349,Forest,636,460,161,15
7,G15,San Roque,Playa Rica,6.4045,-75.0143,1325,Forest,874,645,224,5
8,G17,Alejandria,El Respaldo,6.3978,-75.0444,1389,Non- forest,870,627,235,8
9,G19,Alejandria,El Respaldo,6.4075,-75.0308,1376,Non- forest,886,644,236,6


In [56]:
# Crear un diccionario para asignar colores a cada cluster
colores_clusters = {
    "Cluster 1": 'blue',
    "Cluster 2": 'green',
    "Cluster 3": 'red',
}
# Coordenadas del punto

lat = 6.4137
lng = -75.0366

# Crear mapa centrado en las coordenadas
m = folium.Map(location=[lat, lng], zoom_start=12)

# Agregar círculo azul al mapa
for i in df_grabadoras_cluster.index:
  #Ubicacion grabadora
  lat,lng=df_grabadoras_cluster["Latitud"][i],df_grabadoras_cluster["Longitud"][i]
  tooltip = df_grabadoras_cluster.columns[8]
  radio= int(df_grabadoras_cluster["Cluster 1"][i]/df_grabadoras_cluster["Total_grabaciones"][i]*1*1200)
  folium.Circle(location=[lat, lng], radius=radio, color=colores_clusters["Cluster 1"], fill=True, fill_opacity=0.6, tooltip=tooltip).add_to(m)
  tooltip = df_grabadoras_cluster.columns[9]
  radio= int(df_grabadoras_cluster["Cluster 2"][i]/df_grabadoras_cluster["Total_grabaciones"][i]*1*1200)
  folium.Circle(location=[lat, lng], radius=radio, color=colores_clusters["Cluster 2"], fill=True, fill_opacity=0.6, tooltip=tooltip).add_to(m)
  tooltip = df_grabadoras_cluster.columns[10]
  radio= int(df_grabadoras_cluster["Cluster 3"][i]/df_grabadoras_cluster["Total_grabaciones"][i]*1*1200)
  folium.Circle(location=[lat, lng], radius=radio, color=colores_clusters["Cluster 3"], fill=True, fill_opacity=0.6, tooltip=tooltip).add_to(m)


m.save("/content/drive/MyDrive/Bio_Acustica/mapa_grabadoras_cluster.html")
m


#**Pruebas**

In [ ]:
import numpy as np
spec = np.array([1025, 5168])
spec2 = np.repeat(spec, 10, axis=0)
spec2

array([1025, 1025, 1025, 1025, 1025, 1025, 1025, 1025, 1025, 1025, 5168,
       5168, 5168, 5168, 5168, 5168, 5168, 5168, 5168, 5168])

In [ ]:
import numpy as np
ponderado = np.zeros([1025, 5184])
print(ponderado[0, 0].shape)
for iter, i in enumerate([8]*120):
    spec = np.random.rand(1025, 5184)
    print(iter+1)
    ponderado = (ponderado + spec);
    print(ponderado)


()
1
[[0.21596319 0.2887344  0.57023502 ... 0.57583088 0.67438983 0.11036149]
 [0.18210876 0.9117855  0.30163368 ... 0.94482952 0.9417229  0.94440756]
 [0.26034122 0.23141239 0.86634176 ... 0.8807247  0.95608259 0.64881561]
 ...
 [0.85374788 0.50681098 0.18088635 ... 0.0081807  0.25169575 0.99853413]
 [0.63450102 0.48106174 0.13053612 ... 0.95980793 0.9127217  0.89894023]
 [0.97524638 0.41247035 0.42737127 ... 0.78882914 0.37381059 0.87774196]]
2
[[0.83679497 0.99966094 1.0949052  ... 1.18907635 1.3647818  0.20868536]
 [1.17327703 1.32297734 0.98642235 ... 1.31718451 1.90462681 1.12580417]
 [0.89432481 1.22138676 1.70002213 ... 1.25236976 1.77962192 0.77709387]
 ...
 [1.74895319 1.09882327 0.98562099 ... 0.02677556 1.04247619 1.00442066]
 [0.95454648 1.40204181 0.57341339 ... 1.00654935 1.11991767 1.63441166]
 [1.508627   0.8638744  1.37709044 ... 1.74319693 1.09118295 1.26177892]]
3
[[1.05790478 1.3075631  1.40386846 ... 1.62371133 2.02760811 0.2700158 ]
 [1.19602796 1.46369312 1.2605

In [ ]:
ponderado = ponderado/120
ponderado_g1 #guardar con pickle, quedan 31 pickles 

array([[0.50548416, 0.52635079, 0.49571897, ..., 0.48693152, 0.50699173,
        0.47113582],
       [0.49843304, 0.52018998, 0.47648431, ..., 0.48323596, 0.50301429,
        0.51036017],
       [0.51762077, 0.45591333, 0.49929112, ..., 0.53532806, 0.491451  ,
        0.49487756],
       ...,
       [0.45174386, 0.50023891, 0.49750193, ..., 0.44585653, 0.4754754 ,
        0.50029678],
       [0.47786992, 0.49294852, 0.45934953, ..., 0.45789467, 0.51938121,
        0.4784049 ],
       [0.51112391, 0.47120916, 0.55649599, ..., 0.51172879, 0.50065583,
        0.46471949]])